# Case Study of CBCC (Communities)

In [ ]:
import sys
import os

import polars as pl
import pandas as pd
import numpy as np

In [ ]:
sys.path.append(f"../../methods")
sys.path.append(f"../")

In [ ]:
from cbcc_wrapper import CBCC

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
from io_utils import get_accuracy, get_recall

In [ ]:
from scenario import create_ai_dataset

## Case 1
face, homo, r=5, AI_acc=mean, num_ai=15, M=4

In [ ]:
gt = pd.read_csv("../human_responses/face_gt.csv")
biased_index = gt[gt["gt"]==0]["task"].unique()
gt = gt.set_index("task")

In [ ]:
# make the data
human = pd.read_csv("../human_responses/face_r=5.csv")

In [ ]:
for run in range(15):
    tmp_df = pd.read_csv(f"../ai_responses/face_r=5_ai=mean_target=1_run={run}.csv")
    if run == 0:
        ai_df = tmp_df
    else:
        ai_df = pd.concat([ai_df, tmp_df], ignore_index=True)

In [ ]:
df = pd.concat([human, ai_df], ignore_index=True)

In [ ]:
cbcc = CBCC(labels=[0,1,2,3], C=4)

In [ ]:
ret = cbcc.fit_predict(df, seed=12347)

In [ ]:
acc = get_accuracy(ret, gt)
acc

In [ ]:
recall = get_recall(ret, gt, biased_index)
recall

In [ ]:
workers = df["worker"].unique().tolist()

In [ ]:
with open("../../methods\CBCC\Results\communities.csv", "r") as f:
    communities_csv = f.read()

In [ ]:
prob = {}
worker_index = -1
for line in communities_csv.split("\n")[:-2]:
    if " " in line:
        try:
            values = list(map(float, line.split(" ")[:-1]))
            prob[worker_index] = np.array(values)
        except ValueError as e :
            pass
    else:
        worker_index = int(line)

In [ ]:
human_wokers = human["worker"].unique().tolist()

In [ ]:
cluster_human_count = np.zeros(4)
cluster_AI_count = np.zeros(4)
for i, w in enumerate(workers):
    if w in human_wokers:
        cluster_human_count[prob[i].argmax()] += 1
    else:
        cluster_AI_count[prob[i].argmax()] += 1

In [ ]:
# Community Dist of Humans
cluster_human_count

In [ ]:
# Community Dist of AIs
cluster_AI_count

## Case 2
dog, hetero, r=5, AI_acc=mean, num_ai=30, M=8

In [ ]:
gt = pd.read_csv("../human_responses/dog_gt.csv")
biased_index = gt[gt["gt"]==0]["task"].unique()
gt = gt.astype(int).set_index("task")

In [ ]:
human = pd.read_csv("../human_responses/dog_r=5.csv", dtype={"task":str})

In [ ]:
gen = create_ai_dataset("hetero", "dog", r=5, ai_acc="mean", n_classes=4)

In [ ]:
for ai_df, num_ai in gen:
    if num_ai == 30:
        break

In [ ]:
df = pd.concat([human, ai_df], ignore_index=True)

In [ ]:
cbcc = CBCC(labels=[0,1,2,3], C=8)

In [ ]:
ret = cbcc.fit_predict(df, seed=12347)

In [ ]:
ret.index = ret.index.astype(int)

In [ ]:
acc = get_accuracy(ret, gt)
acc

In [ ]:
recall = get_recall(ret, gt, biased_index.astype(int))
recall

In [ ]:
workers = df["worker"].unique().tolist()

In [ ]:
with open("../../methods\CBCC\Results\communities.csv", "r") as f:
    communities_csv = f.read()

In [ ]:
prob = {}
worker_index = -1
for line in communities_csv.split("\n")[:-2]:
    if " " in line:
        try:
            values = list(map(float, line.split(" ")[:-1]))
            prob[worker_index] = np.array(values)
        except ValueError as e :
            pass
    else:
        worker_index = int(line)

In [ ]:
human_wokers = human["worker"].unique().tolist()

In [ ]:
cluster_human_count = np.zeros(8)
cluster_AI_count = np.zeros(8)
for i, w in enumerate(workers):
    if w in human_wokers:
        cluster_human_count[prob[i].argmax()] += 1
    else:
        cluster_AI_count[prob[i].argmax()] += 1

In [ ]:
# Community Dist of Humans
cluster_human_count

In [ ]:
# Community Dist of AIs
cluster_AI_count